In [1]:
#Import libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
from libsvm.svmutil import svm_train, svm_predict

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [2]:
# data                                                            Shape

hyperberlinnn = loadmat('hyperberlinnn.mat')                      
ch_data = hyperberlinnn['InputImage1D']                     #(90000*114)
ch_result = hyperberlinnn['GroundTruthImage1D']             #(90000*1)
Tr_data = hyperberlinnn['Train']                            #(300*300)
Tr_result = hyperberlinnn['Train1D']                        #(90000*1)
groundTruth = hyperberlinnn['GroundTruth']

In [3]:
# Train and Test(per class) and standardization

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

image1 = StandardScaler().fit_transform(ch_data)
image2 = StandardScaler().fit_transform(ch_result)


X_train, X_test, y_train, y_test = train_test_split(image1 , image2 , test_size = 0.30 , train_size = 0.70 , random_state = 42)

print(np.shape(X_train),np.shape(X_test),np.shape(y_train),np.shape(y_test))


(62999, 114) (27000, 114) (62999, 1) (27000, 1)


In [4]:
selected_features = [  5 ,  8 ,  9 , 16 , 17,  18 , 19,  20 , 21 , 59 , 60 , 61 , 62 , 63 ,110 ,113]

In [5]:
X_train = X_train[:,selected_features]
X_test = X_test[:,selected_features]

In [6]:
# Fitting SVM regression 
# kernel rbf

my_param_grid = {'C': [1,10,30], 'gamma': [1,0.1,0.01], 'kernel': ['rbf']} 

from sklearn.model_selection import GridSearchCV

GridSearchCV(estimator=SVR(),param_grid= my_param_grid, refit = True, verbose=3, cv=2 )



GridSearchCV(cv=2, estimator=SVR(),
             param_grid={'C': [1, 10, 30], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['rbf']},
             verbose=3)

In [7]:
grid = GridSearchCV(estimator=SVR(),param_grid= my_param_grid, refit = True, verbose=2, cv=2 )
# verbose just means the text output describing the process. (the greater the number the more detail you will get).

In [8]:
grid.fit(X_train,y_train)

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time= 1.6min
[CV] END ...........................C=1, gamma=1, kernel=rbf; total time= 1.9min
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time= 1.3min
[CV] END .........................C=1, gamma=0.1, kernel=rbf; total time= 1.2min
[CV] END ........................C=1, gamma=0.01, kernel=rbf; total time= 1.4min
[CV] END ........................C=1, gamma=0.01, kernel=rbf; total time= 1.4min
[CV] END ..........................C=10, gamma=1, kernel=rbf; total time= 3.8min
[CV] END ..........................C=10, gamma=1, kernel=rbf; total time= 4.0min
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time= 1.4min
[CV] END ........................C=10, gamma=0.1, kernel=rbf; total time= 1.4min
[CV] END .......................C=10, gamma=0.01, kernel=rbf; total time= 1.4min
[CV] END .......................C=10, gamma=0.01,

GridSearchCV(cv=2, estimator=SVR(),
             param_grid={'C': [1, 10, 30], 'gamma': [1, 0.1, 0.01],
                         'kernel': ['rbf']},
             verbose=2)

In [9]:
grid.best_params_

{'C': 10, 'gamma': 1, 'kernel': 'rbf'}

In [10]:
grid.best_estimator_

SVR(C=10, gamma=1)

In [11]:
grid.score(X_test, y_test)

0.9302433861089435

In [12]:
#RMSE & R2

predict_train= grid.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,predict_train)))
print(r2_score(y_train,predict_train))


predict_test= grid.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,predict_test))) 
print(r2_score(y_test, predict_test))

0.23069386440160602
0.9466928989698278
0.2646232517348501
0.9302433861089435
